머신 러닝을 이용한 자연어 처리의 최종 목표는 기계가 사람 이상의 성능을 내는 것을 기대하는 것입니다.  
그런데 기계에게 아무리 많은 단어를 학습시켜도, 세상의 모든 단어를 알려줄 수는 없는 노릇입니다.  
그리고 더 많은 단어를 알려주려고 하면 그만큼 계산 비용도 늘어난다는 부담이 있습니다.  

기계가 훈련 단계에서 학습한 단어들의 집합을 단어 집합(vocabulary)이라고 합니다.   
그리고 테스트 단계에서 기계가 미처 배우지 못한 모르는 단어가 등장하면   
그 단어를 단어 집합에 없는 단어란 의미에서 **OOV(Out-Of-Vocabulary)** 또는 **UNK(Unknown Token)** 라고 표현합니다.   
기계가 문제를 풀 때, 모르는 단어가 등장하면 (사람도 마찬가지지만) 주어진 문제를 푸는 것이 훨씬 어려워집니다.  
이와 같이 모르는 단어로 인해 문제를 제대로 풀지 못하는 상황을 OOV 문제라고 합니다.  
  
**단어 분리(Subword segmenation)** 작업은 하나의 단어는 (단어보다 작은 단위의) 의미있는 여러 내부 단어들(subwords)의 조합으로 구성된 경우가 많기 때문에,   
하나의 단어를 여러 내부 단어로 분리해서 단어를 이해해보겠다는 의도를 가진 전처리 작업입니다.  
실제로, 언어의 특성에 따라 영어권 언어나 한국어는 단어 분리를 시도했을 때 어느정도 의미있는 단위로 나누는 것이 가능합니다.  
이 책에서는 이런 작업을 하는 토크나이저를 **단어 분리 토크나이저** 라고 명명하겠습니다.  
  
단어 분리는 기계가 아직 배운 적이 없는 단어에 대해 어느 정도 대처할 수 있도록 하며,  
기계 번역 등에서 주요 전처리로 사용되고 있습니다.   
지금부터 OOV 문제를 완화하는 대표적인 단어 분리 토크나이저인 **BPE(Byte Pair Encoding) 알고리즘** 과   
실무에서 사용할 수 있는 단어 분리 토크나이저 구현체인 **센텐스피스(Sentencepiece)**를 소개합니다.

# 1. BPE(Byte Pair Encoding)

BPE(Byte pair encoding) 알고리즘은 1994년에 제안된 데이터 압축 알고리즘입니다.  
하지만 후에 자연어 처리의 단어 분리 알고리즘으로 응용되었는데, 자연어 처리에 어떻게 응용되었는지는 뒤에서 언급하도록 하고,    
우선 기존의 BPE의 작동 방법에 대해서 이해해보겠습니다.   
아래와 같은 문자열이 주어졌을 때, BPE을 수행한다고 해봅시다.    
  
aaabdaaabac    

BPE은 기본적으로 연속적으로 가장 많이 등장한 글자의 쌍을 찾아서 하나의 글자로 병합하는 방식을 수행합니다.  
태생이 압축 알고리즘인 만큼, 여기서는 글자 대신 바이트(byte)라는 표현을 사용하겠습니다.  
예를 들어 위의 문자열 중 가장 자주 등장하고 있는 바이트의 쌍(byte pair)은 'aa'입니다.  
이 'aa'라는 바이트의 쌍을 하나의 바이트인 'Z'로 치환해보겠습니다.   
  
ZabdZabac  
Z=aa  

이제 위 문자열 중에서 가장 많이 등장하고 있는 바이트의 쌍은 'ab'입니다. 이제 이 'ab'를 'Y'로 치환해봅시다.
  
ZYdZYac  
Y=ab   
Z=aa      

이제 가장 많이 등장하고 있는 바이트의 쌍은 'ZY'입니다. 이를 'X'로 치환해봅시다.  
XdXac  
X=ZY  
Y=ab  
Z=aa  

이제 더 이상 병합할 바이트의 쌍은 없으므로 BPE는 위의 결과를 최종 결과로 하여 종료됩니다.  

# 2. 자연어 처리에서의 BPE(Byte Pair Encoding)

논문 : https://arxiv.org/pdf/1508.07909.pdf  
  
자연어 처리에서의 BPE는 단어 분리(word segmentation) 알고리즘입니다. 기존에 있던 단어를 분리한다는 의미입니다.   
BPE을 요약하면, 글자(charcter) 단위에서 점차적으로 단어 집합(vocabulary)을 만들어 내는 Bottom up 방식의 접근을 사용합니다.  
우선 훈련 데이터에 있는 단어들을 모든 글자(chracters) 또는 유니코드(unicode) 단위로 단어 집합(vocabulary)를 만들고,  
가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합합니다.  
  
BPE을 자연어 처리에 사용한다고 제안한 논문(Sennrich et al. (2016))에서 이미 BPE의 코드를 공개하였기 때문에,   
바로 파이썬 실습이 가능합니다. 코드 실습을 진행하기 전에 육안으로 확인할 수 있는 간단한 예를 들어보겠습니다.  

# 2. WPM(Wordpiece Model)

WPM의 아이디어를 제시한 논문 : https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf  
구글이 위 WPM을 변형하여 번역기에 사용했다는 논문 : https://arxiv.org/pdf/1609.08144.pdf  
    
기존의 BPE 외에도 WPM(Wordpiece Model)이나 Unigram Language Model Tokenizer와 같은 단어 분리 토크나이저들이 존재합니다.    
여기서는 WPM에 대해서만 간략히 언급합니다. 구글은 WPM을 일종의 BPE의 변형으로 소개합니다.     
WPM은 BPE과는 달리 빈도수가 아니라 우도(likelihood)를 통해서 단어를 분리합니다. 여기서는 자세한 설명은 생략하겠습니다.    
2016년에 논문(위 링크 참고)에서 구글은 자신들의 구글 번역기에서 WPM이 수행된 결과에 대해서 기술하였습니다.    
  
WPM을 수행하기 이전의 문장: Jet makers feud over seat width with big orders at stake  
WPM을 수행한 결과(wordpieces): _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake  
  
Jet는 J와 et로 나누어졌으며, feud는 fe와 ud로 나누어진 것을 볼 수 있습니다.   
WPM은 모든 단어의 맨 앞에 _를 붙이고, 단어는 내부단어(subword)로 통계에 기반하여 띄어쓰기로 분리합니다.   
여기서 언더바 _는 문장 복원을 위한 장치입니다. 예컨대, WPM의 결과로 나온 문장을 보면,   
Jet → _J et와 같이 기존에 없던 띄어쓰기가 추가되어 내부 단어(subwords)들을 구분하는 구분자 역할을 하고 있습니다.   
그렇다면 기존에 있던 띄어쓰기와 구분자 역할의 띄어쓰기는 어떻게 구별할까요? 이 역할을 수행하는 것이 단어들 앞에 붙은 언더바 _입니다.  
WPM이 수행된 결과로부터 다시 수행 전의 결과로 돌리는 방법은 현재 있는 모든 띄어쓰기를 전부 제거하고, 언더바를 띄어쓰기로 바꾸면 됩니다.  
  
구글의 WPM은 오픈소스로 공개되지 않았습니다.  

# 3. 센텐스피스(Sentencepiece)

논문 : https://arxiv.org/pdf/1808.06226.pdf  
센텐스피스 깃허브 : https://github.com/google/sentencepiece  
  
결론적으로 실무에서 단어 분리를 위해서 어떤 구현체를 사용해야 하냐고 묻는다면, 구글의 센텐스피스(Sentencepiece)를 사용합니다.  
구글은 BPE 알고리즘과 Unigram Language Model Tokenizer를 구현한 센텐스피스를 깃허브에 공개하였습니다.   
기존의 BPE 알고리즘 논문 저자 또한 BPE 코드를 깃허브에 공개하기는 했지만,   
이를 실무에 사용하기에는 속도가 매우 느리므로 센텐스피스를 사용하는 것을 권장합니다.  

센텐스피스의 이점은 또 있습니다.   
단어 분리 알고리즘을 사용하기 위해서, 데이터에 단어 토큰화를 먼저 진행한 상태여야 한다면   
이 단어 분리 알고리즘을 모든 언어에 사용하는 것은 쉽지 않습니다. 영어와 달리 한국어와 같은 언어는 단어 토큰화부터가 쉽지 않기 때문입니다.  
그런데, 이런 사전 토큰화 작업(pretokenization)없이 전처리를 하지 않은 데이터(raw data)에 바로 단어 분리 토크나이저를 사용할 수 있다면,  
이 토크나이저는 그 어떤 언어에도 적용할 수 있는 토크나이저가 될 것입니다.  
센텐스피스는 이 이점을 살려서 구현되었습니다.   
센텐스피스는 사전 토큰화 작업없이 단어 분리 토큰화를 수행하므로 언어에 종속되지 않습니다.  

# Reference

BPE 알고리즘 논문 : https://arxiv.org/pdf/1508.07909.pdf  
BPE 알고리즘 논문 저자의 깃허브 : https://github.com/rsennrich/subword-nmt  
서브워드 알고리즘 비교 : https://medium.com/@makcedward/how-subword-helps-on-your-nlp-model-83dd1b836f46  
WPM의 아이디어를 제시한 논문 : https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf  
WPM을 사용한 구글의 번역기에 대한 논문 : https://arxiv.org/pdf/1609.08144.pdf  
WPM 참고 자료 : https://norman3.github.io/papers/docs/google_neural_machine_translation.html  
유니그램 언어 모델을 이용한 단어 분리 : https://arxiv.org/pdf/1804.10959.pdf  
센텐스피스 사용한 한국어 실습 참고 자료 : https://bab2min.tistory.com/622  
wordpiece Vs. sentencepiece : https://mc.ai/pre-training-bert-from-scratch-with-cloud-tpu   
https://mlexplained.com/2019/11/06/a-deep-dive-into-the-wonderful-world-of-preprocessing-in-nlp/  